This problem attempts to improve the approximation obtained by rounding a linear program for vertex cover. As described in [WS], this is a special case of the set cover problem. In the lecture we looked at a rounding-based approximation algorithm, and noticed that it was a factor at most 2 from an optimal solution (see below).

    * Argue that for every triangle in the graph, it is valid to add an extra constraint requiring at least two vertices to be selected, in the sense that it does not remove any integer feasible solution.

    * Write code to add these constraints to the existing ones. You can modify the existing Python code, or build your own implementation from scratch in another language.
    
    * Compare the rounded solutions, as well as the lower bounds on the optimal solution, to the original implementation.

You will need to implement an algorithm for computing the triangles in a graph, e.g., you could use the "trivial algorithm" described on page 1 of [this paper]. Note that the graphs supplied are undirected, but each edge {u, v} only appears once, either as (u,v) or as (v,u). (Extra credit: Try to make the algorithm scalable in terms of the number of edges and vertices.)

You must hand in a PDF with all explanation and code. Please also provide a link to your implementation (e.g. on colab or github).

In [8]:
# Fetch and import APX wrapper class
!wget -q https://raw.githubusercontent.com/rasmus-pagh/apx/main/apx.py -O apx.py
import apx
from importlib import reload
reload(apx)
from apx import DataFile, LinearProgram, np

## Solving a linear program and its dual

In [10]:
mylp = LinearProgram('max')
mylp.add_constraint({"x1": 4, "x2": 1}, 2)
mylp.add_constraint("x1 + 2*x2", 1)
mylp.set_objective({"x1": 1, "x2": 1})
print(mylp.to_string())
print(mylp.solve())
dual = mylp.dual()
print(dual.to_string())
print(dual.solve())
print(dual.dual().to_string())
print(dual.dual().solve())

Maximize c x under A x <= b, x >= 0, where
A=[[4. 1.]
 [1. 2.]]
b=[2, 1]
c=[1, 1]
(0.7142857142857144, {'x1': 0.42857142857142855, 'x2': 0.2857142857142858})
Minimize b y under A y >= c, y >= 0, where
A=[[4. 1.]
 [1. 2.]]
b=[2, 1]
c=[1, 1]
(0.7142857142857143, {'y1': 0.14285714285714285, 'y2': 0.4285714285714286})
Maximize c x under A x <= b, x >= 0, where
A=[[4. 1.]
 [1. 2.]]
b=[2, 1]
c=[1, 1]
(0.7142857142857144, {'x1': 0.42857142857142855, 'x2': 0.2857142857142858})


# Approximating vertex cover

In [74]:
def trivial_triangle_enum(graph):
  edges = list(graph)
  triangles = set()
  covered_edges = set()
  for (u,v) in edges:
    for (up, vp) in edges:
      if up in [u, v] or vp in [u, v]:
        if (up, vp) != (u,v):  
          for (upp, vpp) in edges:
            if (upp in [u, v, up, vp]) and (vpp in [u, v, up, vp]):
              if (upp, vpp) not in ((up, vp), (u,v)):
                  if not ((u,v) in covered_edges and (up,vp) in covered_edges and (upp,vpp) in covered_edges):
                    triangles.add(((u,v),(up,vp),(upp,vpp)))
                    covered_edges.add((u,v))
                    covered_edges.add((v,vp))
                    covered_edges.add((vp,u))
  return triangles
    

for filename in DataFile.graph_files:
  graph = DataFile(filename)
  graph_copy = DataFile(filename)
  triangles = trivial_triangle_enum(graph_copy)
  vertex_cover_lp = LinearProgram('min')
  objective = {}
  for (u,v) in graph:
    vertex_cover_lp.add_constraint({u: 1, v: 1}, 1)
    objective[u] = 1.0
    objective[v] = 1.0
  vertex_cover_lp.set_objective(objective)
  value, solution = vertex_cover_lp.solve()
  rounded_value, rounded_solution = 0, {}
  for x in solution:
    r = int(np.round(solution[x] + 1e-10))
    # Add small constant to deal with numerical issues for numbers close to 1/2
    rounded_solution[x] = r
    rounded_value += r

  print(f'Graph: {filename}')
  print(f'LP optimal value: {value}\n{solution}')
  print(f'Rounded LP value: {rounded_value}\n{rounded_solution}')
  
  if triangles:
    print(f"\n{len(triangles)} TRIANGLE(S):")  
    for ((u,v),(v,x),(x,u)) in triangles:
      vertex_cover_lp.add_constraint({u: 1, v: 1, x: 1}, 2)
    value, solution = vertex_cover_lp.solve()
    rounded_value, rounded_solution = 0, {}
    for x in solution:
      r = int(np.round(solution[x] + 1e-10))
      # Add small constant to deal with numerical issues for numbers close to 1/2
      rounded_solution[x] = r
      rounded_value += r
    print(f'LP optimal value, triangles: {value}\n{solution}')
    print(f'Rounded LP value, triangles: {rounded_value}\n{rounded_solution}\n')

Graph: routes.txt
LP optimal value: 5.0
{'JFK': 0.5, 'MCO': 0.5, 'ORD': 0.5, 'DEN': 0.5, 'HOU': 0.5, 'DFW': 0.5, 'PHX': 0.5, 'ATL': 0.5, 'LAX': 0.5, 'LAS': 0.5}
Rounded LP value: 10
{'JFK': 1, 'MCO': 1, 'ORD': 1, 'DEN': 1, 'HOU': 1, 'DFW': 1, 'PHX': 1, 'ATL': 1, 'LAX': 1, 'LAS': 1}

27 TRIANGLE(S):
LP optimal value, triangles: 8.0
{'JFK': 1.0, 'MCO': -0.0, 'ORD': 1.5, 'DEN': 0.5, 'HOU': 1.0, 'DFW': 0.5, 'PHX': 0.5, 'ATL': 1.0, 'LAX': 0.5, 'LAS': 1.5}
Rounded LP value, triangles: 11
{'JFK': 1, 'MCO': 0, 'ORD': 2, 'DEN': 1, 'HOU': 1, 'DFW': 1, 'PHX': 1, 'ATL': 1, 'LAX': 1, 'LAS': 2}

Graph: petersen.txt
LP optimal value: 5.0
{'A': 0.5, 'B': 0.5, 'C': 0.5, 'D': 0.5, 'E': 0.5, '1': 0.5, '3': 0.5, '5': 0.5, '2': 0.5, '4': 0.5}
Rounded LP value: 10
{'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, '1': 1, '3': 1, '5': 1, '2': 1, '4': 1}
Graph: petersenstar.txt
LP optimal value: 6.0
{'A': 0.5, 'B': 0.5, 'C': 0.5, 'D': 0.5, 'E': 0.5, '1': 0.5, '3': 0.5, '5': 0.5, '2': 0.5, '4': 0.5, '10': 1.0, '11': 0.